# Django Configuration and setup #
I used the Anaconda base environment and installed Django creating a smartnotes project with ` django-admi startproject smartnotes ` n , an app within (webpage) called home and the functions of that app in the views.py file which is created with the ` django-admin startapp [app] `

To start the Django server for testing use ` python manage.py runserver `

## The path ##
<img src='assets/ThePath.jpg'>

<img src='assets/MVTFramework.jpg'>

The Model View Template.  The model layer handles the data and how it's stored.  The view is where we employ the functions to work with the data and the template is how it will render in HTML.  Django Template Language is what the template employs as code in the view layer is passed down to the template layer and that's what makes the html in a manner similar to a JavaScript insertion.  DTL allows for rich HTML formatting without a lot of HTML coding. 

## Migrations ##
To keep databases in sync changes are maintained in a `migrations` folder and a `python manage.py migrate` command will synchronize these in the database. 

<img src='assets/Migrations.jpg'>

# The Django Admin Page #

` /admin ` at the localhost URL for the python server will take you to the Django Admin page.

First thing is to create a super user with the `python manage.py createsuperuser` command. 

With the user you can use the built in admin page to configure users and group permissions to create and delete accounts, contents or modify privileges.  The `@login_required(login_url='/admin')` appearing in the views.py file will prevent following functions from running unless logged in with correct privilege and will redirect to the login URL supplied.  One line to secure code with a robust authentication system including encrypted passwords. 

# Django Object Relational Mapping System #

Class models get written then migrated into database tables where the class defines the table and the class attributes are columns in the database table. 

<img src=assets/ClassMapping.jpg>

This system is particularly useful for those who wish to use Python to script web applications using SQL databases.

For instance to create the database for a note taking app we'd want a database called *Notes* and the code that makes lives in the Django app folders' `models.py` file as: 
```
class Notes(models.Model):
    title = models.CharField(max_length=200)
    text = models.TextField()
    created = models.DateTimeField(auto_now_add=True)

```

And then a run of the `python manage.py makemigrations` would convert that to a script that upon next migration will create database table.  Then do `python manage.py migrate

When an app is initiated with django-admin startapp a folder is created and populated with other folders and files including an `admin.py` folder in which you must indicate where the file of models you wish to use are.  The models.py folder has the classes you wish to use in your app. `

## Django Shell ##

You can use this tool to check the contents of a database `python manage.py shell`  it constructs a command line interface with an awareness of the project and database so you can directly query and even add to the database from the command line. 
`Notes.objects.all()` will return a list of all the objects in the database called Notes.  Other methods in the database objects available will filter including or excluding content. 

# Base Template #
Code you want on all the rendered HTML pages should appear in a base template rather than adding it to each template that generates a specific page for your site.  Django is accustomed to finding such a template (called `base.html`) in a template folder under a static folder, where' you'd also put CSS and other assets to be made available to all your rendered pages.

Here is an example base.html template:
```
{% load static %}
<html>
    <head>
        <link rel="stylesheet" type="text/css" href="{% static 'css/style.css' %}" />
    </head>

    <body>
        {% block content %}
        {% endblock %}
    </body>
</html>
```
The assumption here is that there's a stylesheet named style.css in a css folder under static and that we desire an injectable object called `content` to which we'll direct the other templates. Those are pointed here with 
`{% extends "base.html" %}` loaders at the top of each other template. 

It's also important to add the (in this case `static`) folder to the list in the `settings.py` folder where there's a premade list `TEMPLATES` with a sublist for `DIRS` that comes empty. This would be modified as below: 

```
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [
            BASE_DIR / 'static/templates',
        ],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]
```

So a template that will leverage a base template already configured and just extend other elements of content as well would be coded as: 
```
{% extends "base.html" %}

{% block content %}
    <h1>These are the notes:</h1>

    <ul> 
        {% for note in notes %}
            <li class="note-li">{{note.title}}</li>
        {% endfor %}
    </ul>
{% endblock %}
```

You will add page specifice CSS to each page's template and CSS that is for the whole site would be on the `base.html` template. 


# **C**reate **R**etrieve **U**pdate **D**elete #

The CRUD refers to the minimal operations that a model should support.  This involves importing the addtional methods such as CreateView being imported and functions build in the `view.py` and the URL updated in the `urls.py` files. Finally the HTML elements you'll want on the page rendered as part of each operation needs to be created in the `templates` folder.

For creating content you'll likely want a form.  There's a nuance in Django to prevent improper form submission by requiring a token, called a cross site request forgery token,  with each form to allow Django to ensure the right form is being acted upon. Here's a sample code indicating the call for a token in the action part of the form:
```
<form action="{% url 'notes.new' %}" method='POST'>{% csrf_token %}
    {{ form }}
    <button type="submit" class="btn btn-primary my-5">Submit</button>
```

The token is unique to the browser session so even hijacked credentials can't use another browser to intercept the form. 

<img src='assets/CSRF01.jpg'>
<img src='assets/CSRF02.jpg'>
<img src='assets/CSRF03.jpg'>

# Model Forms #

A form template that provides the post action (with the anti cross site token) and error handling appears below:
```
<form method='POST'>{% csrf_token %}
    {{ form.as_p }}
    <button type="submit" class="btn btn-primary my-5">Submit</button>
    <a href="{% url 'notes.list' %}" class="btn btn-secondary">Cancel</a>
</form>
{% if form.errors %}
<div class="alert alert-danger my-5">
    {{form.errors.title.as_text}}
</div>
{% endif %}
```

This would be paired with the form model `forms.py` : 
```
from django import forms
from django.core.exceptions import ValidationError
from .models import Notes


class NotesForm(forms.ModelForm):
    class Meta:
        model = Notes
        fields = ('title', 'text')
        widgets = {
            'title': forms.TextInput(attrs={'class': 'form-control my-5'}),
            'text': forms.Textarea(attrs={"class": "form-control mb-5"})
        }
        labels = {
            'text': 'Write your thoughts here:'
        }

    def clean_title(self):
        title = self.cleaned_data['title']
        if 'Django' not in title:
            raise ValidationError('We only accept notes about Django!')
        return title
```

and the form view classes that provide the CRUD functions
```
rom typing import List
from django.shortcuts import render
from django.http import Http404
from django.views.generic import CreateView, DetailView, ListView, UpdateView
from django.views.generic.edit import DeleteView

from .forms import NotesForm
from .models import Notes

class NotesDeleteView(DeleteView):
    model = Notes
    success_url = '/smart/notes'
    template_name = "notes/notes_delete.html"
class NotesUpdateView(UpdateView):
    model = Notes
    success_url = '/smart/notes'
    form_class = NotesForm
class NotesCreateView(CreateView):
    model = Notes
    success_url = '/smart/notes'
    form_class = NotesForm

class NotesListView(ListView):
    model = Notes
    context_object_name = "notes"
    template_name = "notes/notes_list.html"

class NotesDetailView(DetailView):
    model = Notes
    context_object_name = "note"
```



# Authentication #